In [14]:
# MASK DETECTOR MODEL

In [1]:
import cv2
import os

# Loading the Data

In [2]:
#location of your dataset

address = "C:/Users/91808/Downloads/SPK IIIT/datasets/"

In [3]:
features = []
target = []
for i in ["WithMask","WithoutMask"]:
    collection_image_names = os.listdir(address + "Face Mask Dataset/Train/" + str(i))
    for j in collection_image_names:
        img = cv2.imread(address + "Face Mask Dataset/Train/" + str(i) + "/" + j)
        try:
            img = cv2.resize(img,(32,32))
        except:
            pass
        else:
            features.append(img)
            if i=="WithMask":
                target.append(0)
            else:
                target.append(1)       
    print("Loaded Folder ",i)

Loaded Folder  WithMask
Loaded Folder  WithoutMask


In [4]:
import numpy as np
features = np.array(features)
target = np.array(target)

In [5]:
def preprocessing(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/225
    return image

In [6]:
features = np.array(list(map(preprocessing,features)))

In [7]:
from sklearn.model_selection import train_test_split
train_features,test_features,train_target,test_target = train_test_split(features,target,test_size=0.2)

In [8]:
train_features.shape

(8000, 32, 32)

In [9]:
train_features = train_features.reshape(8000,32,32,1)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
dataGen = ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)
dataGen.fit(train_features)
batches = dataGen.flow(train_features,train_target,batch_size=20)
images,labels = next(batches)


In [12]:
images.shape

(20, 32, 32, 1)

In [13]:
images = images.reshape(20,32,32)

# Model

In [14]:
from tensorflow.keras.utils import to_categorical
train_target = to_categorical(train_target)
train_target.shape


(8000, 2)

# Architecture

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,Dropout

In [16]:
model = Sequential()
model.add(Conv2D(100, (3,3), activation='relu', input_shape = (32,32,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(30,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))

# Compile

In [17]:
from tensorflow.keras.optimizers import Adam
model.compile(Adam(learning_rate=0.001),loss = "categorical_crossentropy",metrics=["accuracy"])

# Train

In [18]:
model.fit(train_features,train_target,epochs=20,validation_split = 0.2)

Epoch 1/20
200/200 [==============================] - 35s 167ms/step - loss: 0.3729 - accuracy: 0.8152 - val_loss: 0.2330 - val_accuracy: 0.9069
Epoch 2/20
200/200 [==============================] - 34s 169ms/step - loss: 0.1931 - accuracy: 0.9291 - val_loss: 0.1575 - val_accuracy: 0.9525
Epoch 3/20
200/200 [==============================] - 35s 173ms/step - loss: 0.1388 - accuracy: 0.9495 - val_loss: 0.0949 - val_accuracy: 0.9712
Epoch 4/20
200/200 [==============================] - 30s 150ms/step - loss: 0.1224 - accuracy: 0.9569 - val_loss: 0.1265 - val_accuracy: 0.9525
Epoch 5/20
200/200 [==============================] - 29s 145ms/step - loss: 0.1098 - accuracy: 0.9581 - val_loss: 0.0738 - val_accuracy: 0.9744
Epoch 6/20
200/200 [==============================] - 29s 147ms/step - loss: 0.0981 - accuracy: 0.9653 - val_loss: 0.0757 - val_accuracy: 0.9725
Epoch 7/20
200/200 [==============================] - 29s 145ms/step - loss: 0.0869 - accuracy: 0.9716 - val_loss: 0.0777 - val_ac

# Saving the Model

In [4]:
location = "C:/Users/91808/Downloads/SPK IIIT/internships and courses/python/"

In [ ]:
ModelInJson = model.to_json()
abc = open(location + "files for mask detector model/model_mask_detector2.json","w")
abc.write(ModelInJson)

model.save_weights(location + "files for mask detector model/model_mask_detector2_weights.h5")

# Loading the Model

In [1]:
location = "C:/Users/91808/Downloads/SPK IIIT/internships and courses/python/"

In [2]:
from tensorflow.keras.models import model_from_json
abc = open(location + "files for mask detector model/model_mask_detector2.json","r")
loaded_data = abc.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights(location + "files for mask detector model/model_mask_detector2_weights.h5")

# Testing The Model

In [3]:
def getClassName(classNo):
    if classNo == 0:    return "YES"
    elif classNo == 1:  return "NO"

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

cv2.ocl.setUseOpenCL(False)

capt = cv2.VideoCapture(0)


while True:
    msg,image = capt.read()
    face_cascade = cv2.CascadeClassifier(location + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.3,minNeighbors = 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(image,(x,y-50),(x+w, y+h+10), (255,0,0),2)
        face_image = gray[y:y+h, x:x+w]
        resize = cv2.resize(face_image,(32,32))
        normalized = resize/225.0
        reshaped = np.reshape(normalized,(1,32,32,1))
        prediction = loaded_model.predict(reshaped)
        neuron_index = loaded_model.predict_classes(reshaped)
        
        cv2.putText(image,"Mask: ",(20,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        cv2.putText(image,"Probablity: ",(20,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        ProbabilityValue = np.amax(prediction)
        if ProbabilityValue> 0.25:
            cv2.putText(image,getClassName(neuron_index),(120,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
            cv2.putText(image,str(int(ProbabilityValue*100)) + "%",(200,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    cv2.imshow('MASK DETECTOR', cv2.resize(image,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    